In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

# # 한글 폰트 사용을 위해서 세팅
# from matplotlib import font_manager, rc
# font_path = "C:/Windows/Fonts/H2GTRM.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()

# plt.rc('font', family = font)
# plt.rc('axes', unicode_minus=False)

from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/bigcontest/test_data.csv')
test = data[['base_date','emd_nm','total_pop','제주_resd_ratio','제주_resd_pop']]
test

,base_date,emd_nm,total_pop,제주_resd_ratio,제주_resd_pop
0,2021-07-01,건입동,9108.666042,0.546794,6721.705127
1,2021-07-01,구좌읍,15951.164270,0.744055,13501.923490
2,2021-07-01,남원읍,19349.706370,0.771138,14556.895855
3,2021-07-01,노형동,57461.240850,0.733652,53775.986296
4,2021-07-01,대륜동,15280.093538,0.714609,14125.330274
...,...,...,...,...,...
2599,2021-08-31,표선면,12646.088392,0.792154,10599.763562
2600,2021-08-31,한경면,9514.481035,0.733958,7097.621164
2601,2021-08-31,한림읍,24494.241076,0.752209,20322.779888
2602,2021-08-31,화북동,24578.081764,0.731218,25715.742872


## 예측할 data 가져오기

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/bigcontest/ndf.csv')
df

,datetime,emd_nm,visit_ratio,frgn_ratio,youth_ratio
0,2018-01-01,건입동,49.642958,0.038427,0.200261
1,2018-01-01,남원읍,41.452047,0.025900,0.177984
2,2018-01-01,노형동,25.674217,0.039271,0.195624
3,2018-01-01,대륜동,42.502310,0.014440,0.208504
4,2018-01-01,대정읍,28.652628,0.053508,0.168114
...,...,...,...,...,...
50881,2021-06-30,한경면,35.350145,0.042559,0.171578
50882,2021-06-30,한림읍,35.614095,0.048140,0.197206
50883,2021-06-30,화북동,27.871984,0.010642,0.167519
50884,2021-06-30,효돈동,31.896263,0.014730,0.165783


## visit_ratio

In [ ]:
vr = df[['datetime','emd_nm','visit_ratio']]
vr

,datetime,emd_nm,visit_ratio
0,2018-01-01,건입동,49.642958
1,2018-01-01,남원읍,41.452047
2,2018-01-01,노형동,25.674217
3,2018-01-01,대륜동,42.502310
4,2018-01-01,대정읍,28.652628
...,...,...,...
50881,2021-06-30,한경면,35.350145
50882,2021-06-30,한림읍,35.614095
50883,2021-06-30,화북동,27.871984
50884,2021-06-30,효돈동,31.896263


In [ ]:
strt = '2021-07-01'
end = '2021-08-31'
df1 = pd.DataFrame( columns = ['ds','yhat','emd_nm'])

for i in tqdm(vr.emd_nm.unique()):
      a = vr[vr.emd_nm == i]   

      final_model = Prophet(
                      changepoint_prior_scale= 5,
                      changepoint_range = 0.9,
                      n_changepoints = 200,
                      seasonality_mode = 'additive',
                      seasonality_prior_scale = 0.1,
                      weekly_seasonality= True,
                      yearly_seasonality = True,
                      interval_width=0.95)

      final_model.fit(a.rename(columns = {'datetime':'ds' ,'visit_ratio':'y'}))
      train_forecast = final_model.make_future_dataframe(periods = 90 , freq='D' ,include_history =False)
      train_forecast = final_model.predict(train_forecast)

      n = train_forecast[['ds','yhat']]
      n['emd_nm'] = i  
      a = n[(n.ds >= strt)&(n.ds <= end)]
      df1 = df1.append(a)

  0%|          | 0/42 [00:00<?, ?it/s]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  2%|▏         | 1/42 [00:36<25:15, 36.96s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  

In [ ]:
df1.to_csv("/content/drive/MyDrive/bigcontest/testvisit_ratio.csv")

## frgn_ratio

In [ ]:
vr = df[['datetime','emd_nm','frgn_ratio']]
vr

,datetime,emd_nm,frgn_ratio
0,2018-01-01,건입동,0.038427
1,2018-01-01,남원읍,0.025900
2,2018-01-01,노형동,0.039271
3,2018-01-01,대륜동,0.014440
4,2018-01-01,대정읍,0.053508
...,...,...,...
50881,2021-06-30,한경면,0.042559
50882,2021-06-30,한림읍,0.048140
50883,2021-06-30,화북동,0.010642
50884,2021-06-30,효돈동,0.014730


In [ ]:
strt = '2021-07-01'
end = '2021-08-31'
df2 = pd.DataFrame( columns = ['ds','yhat','emd_nm'])

for i in tqdm(vr.emd_nm.unique()):
      a = vr[vr.emd_nm == i]   

      final_model = Prophet(
                      changepoint_prior_scale= 5,
                      changepoint_range = 0.9,
                      n_changepoints = 200,
                      seasonality_mode = 'multiplicative',
                      seasonality_prior_scale = 0.1,
                      weekly_seasonality= True,
                      yearly_seasonality = True,
                      interval_width=0.95)

      final_model.fit(a.rename(columns = {'datetime':'ds' ,'frgn_ratio':'y'}))
      train_forecast = final_model.make_future_dataframe(periods = 90 , freq='D' ,include_history =False)
      train_forecast = final_model.predict(train_forecast)

      n = train_forecast[['ds','yhat']]
      n['emd_nm'] = i  
      a = n[(n.ds >= strt)&(n.ds <= end)]
      df2 = df2.append(a)

  0%|          | 0/42 [00:00<?, ?it/s]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  2%|▏         | 1/42 [00:28<19:38, 28.75s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  

In [ ]:
df2

,ds,yhat,emd_nm
0,2021-07-01,0.024671,건입동
1,2021-07-02,0.024730,건입동
2,2021-07-03,0.024109,건입동
3,2021-07-04,0.024006,건입동
4,2021-07-05,0.024303,건입동
...,...,...,...
57,2021-08-27,0.090975,한경면
58,2021-08-28,0.090333,한경면
59,2021-08-29,0.090016,한경면
60,2021-08-30,0.092673,한경면


In [ ]:
df2.to_csv("/content/drive/MyDrive/bigcontest/testfrgn_ratio.csv")

## youth_ratio

In [ ]:
vr = df[['datetime','emd_nm','youth_ratio']]
vr

,datetime,emd_nm,youth_ratio
0,2018-01-01,건입동,0.200261
1,2018-01-01,남원읍,0.177984
2,2018-01-01,노형동,0.195624
3,2018-01-01,대륜동,0.208504
4,2018-01-01,대정읍,0.168114
...,...,...,...
50881,2021-06-30,한경면,0.171578
50882,2021-06-30,한림읍,0.197206
50883,2021-06-30,화북동,0.167519
50884,2021-06-30,효돈동,0.165783


In [ ]:
strt = '2021-07-01'
end = '2021-08-31'
df3 = pd.DataFrame( columns = ['ds','yhat','emd_nm'])

for i in tqdm(vr.emd_nm.unique()):
      a = vr[vr.emd_nm == i]   

      final_model = Prophet()

      final_model.fit(a.rename(columns = {'datetime':'ds' ,'youth_ratio':'y'}))
      train_forecast = final_model.make_future_dataframe(periods = 90 , freq='D' ,include_history =False)
      train_forecast = final_model.predict(train_forecast)

      n = train_forecast[['ds','yhat']]
      n['emd_nm'] = i  
      a = n[(n.ds >= strt)&(n.ds <= end)]
      df3 = df3.append(a)

  0%|          | 0/42 [00:00<?, ?it/s]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  2%|▏         | 1/42 [00:02<01:52,  2.75s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  

In [ ]:
df3

,ds,yhat,emd_nm
0,2021-07-01,0.203817,건입동
1,2021-07-02,0.204290,건입동
2,2021-07-03,0.204496,건입동
3,2021-07-04,0.204542,건입동
4,2021-07-05,0.204454,건입동
...,...,...,...
57,2021-08-27,0.181510,한경면
58,2021-08-28,0.183198,한경면
59,2021-08-29,0.183386,한경면
60,2021-08-30,0.183316,한경면


## 합치기

In [ ]:
df1 = df1.rename(columns = {'yhat':'visit_ratio'})
df2 = df2.rename(columns = {'yhat':'frgn_ratio'})
df3 = df3.rename(columns = {'yhat' :'youth_ratio'})

In [ ]:
frgn_ratio = pd.read_csv(("/content/drive/MyDrive/bigcontest/testfrgn_ratio.csv"))
df2 = frgn_ratio[['ds','yhat','emd_nm']].rename(columns = {'yhat':'frgn_ratio'})

In [ ]:
df1 = df1.astype({'ds':'str'})
df1

,ds,visit_ratio,emd_nm
0,2021-07-01,51.067891,건입동
1,2021-07-02,51.689743,건입동
2,2021-07-03,52.075000,건입동
3,2021-07-04,51.074567,건입동
4,2021-07-05,50.473149,건입동
...,...,...,...
57,2021-08-27,31.776870,한경면
58,2021-08-28,34.104621,한경면
59,2021-08-29,33.077186,한경면
60,2021-08-30,28.527242,한경면


In [ ]:
df3 = df3.astype({'ds':'str'})
df3

,ds,youth_ratio,emd_nm
0,2021-07-01,0.203817,건입동
1,2021-07-02,0.204290,건입동
2,2021-07-03,0.204496,건입동
3,2021-07-04,0.204542,건입동
4,2021-07-05,0.204454,건입동
...,...,...,...
57,2021-08-27,0.181510,한경면
58,2021-08-28,0.183198,한경면
59,2021-08-29,0.183386,한경면
60,2021-08-30,0.183316,한경면


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2604 entries, 0 to 2603
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ds          2604 non-null   object 
 1   frgn_ratio  2604 non-null   float64
 2   emd_nm      2604 non-null   object 
dtypes: float64(1), object(2)
memory usage: 61.2+ KB


In [ ]:
new_df = pd.merge(df1, df2 , how = 'inner')

In [ ]:
new = pd.merge(new_df , df3 , how = 'inner')
new = new.rename(columns = {'ds':'base_date'})

In [ ]:
test_set = pd.merge(test , new , how = 'inner')
test_set.isna().sum()

base_date        0
emd_nm           0
total_pop        0
제주_resd_ratio    0
제주_resd_pop      0
visit_ratio      0
frgn_ratio       0
youth_ratio      0
dtype: int64

In [ ]:
test_set.to_csv("/content/drive/MyDrive/bigcontest/test_set_sh.csv")